<a href="https://colab.research.google.com/github/secureterminal/disney_web_scraping/blob/main/Disney_Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Get info Box


In [16]:
from bs4 import BeautifulSoup as bs
import requests

## Load the webpage


In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')

# convert it to a beautiful soup object
soup = bs(r.content)

# print out html

contents = soup.prettify()

In [ ]:
print(contents)

In [ ]:
info_box = soup.find(class_="infobox vevent")

info_rows = info_box.find_all("tr")

for row in info_rows:
    print(row.prettify())

In [ ]:
movie_info = {}

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")
    
for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value



movie_info

## Task 2:  Get info Box for all movies

In [42]:

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")


def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
        
def get_info_box(url):
    r = requests.get(url)
    soup = bs(r.content)  
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    clean_tags(soup)

    movie_info = {} 
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        # elif index == 1:
        #     continue
        else:
            header = row.find("th")
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value

    return movie_info





In [74]:
# Test
toy = get_info_box("https://en.wikipedia.org/wiki/The_Vanishing_Prairie")

In [75]:
toy

{'Box office': '$1.75 million (US and Canadian rentals)',
 'Cinematography': 'N. Paul Kenworthy',
 'Country': 'United States',
 'Directed by': 'James Algar',
 'Distributed by': 'Buena Vista Distribution',
 'Edited by': 'Lloyd L. Richardson',
 'Language': 'English',
 'Music by': 'Paul J. Smith',
 'Narrated by': 'Winston Hibler',
 'Produced by': ['Ben Sharpsteen', 'Walt Disney'],
 'Production company': 'Walt Disney Productions',
 'Release date': ['August 17, 1954'],
 'Running time': ['60 minutes (VHS version)', '71 minutes (original)'],
 'Written by': ['James Algar', 'Winston Hibler'],
 'title': 'The Vanishing Prairie'}

In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# convert it to a beautiful soup object
soup = bs(r.content)
movie_info_list = []
movies = soup.select(".wikitable.sortable i")
base_path =  "https://en.wikipedia.org/"
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print("Scraped " + str(index) + " movies")
    try:
        relative_path = movie.a['href']
        full_path = base_path + relative_path
        title = movie.a['title']
        movie_info_list.append(get_info_box(full_path))
    except Exception as e:
        print(movie.get_text())
        print(e)

In [46]:
len(movie_info_list)

502

Save/Reload Data


In [57]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [58]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f) 
        

In [59]:
save_data("disney_data_3.json", movie_info_list)

In [56]:
movie_info_list = load_data("disney_data_2.json")


#### Subtasks

- ~~Clean up references [1]~~
- Convert running time into an integer
- Convert dates into a datetime object
- ~~Split up the long strings~~
- Convert Budget & Box office to numbers

In [ ]:
count = 0
no_run_time = 0
for a in movie_info_list:
    count+= 1
    if 'Running time' in a.keys():
        if isinstance(a['Running time'], list):
            movie_time = a['Running time'][0].split("min")[0].strip()
            print(a['title'] + " :", a['Running time'][0] + " :", movie_time)
        else:
            movie_time = a['Running time'].split("min")[0].strip()
            print(a['title'] + " :", a['Running time'] + " :", movie_time)
    else:
        no_run_time+=1
    if count > 10:
        continue
    
    # for key, value in a:
    #     if key == 'Running time':
    #         print(key[value])
print(count, no_run_time)